# Tensorflow - Marker Quality Control

In [1]:
import mlflow
import tensorflow as tf

2025-12-03 14:11:05.732935: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/marc-medlock/PycharmProjects/model_platform/.venv/lib/python3.12/site-packages/google/protobuf/runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/attr_value.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
/home/marc-medlock/PycharmProjects/model_platform/.venv/lib/python3.12/site-packages/google/protobuf/runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/tensor.proto. Please update the ge

### Chargement du modèle

In [2]:
model = tf.saved_model.load(
    "marker_quality_control/1", tags=None, options=None
)

2025-12-03 14:11:09.580111: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


## Prédiction sur une image

In [3]:
img_path = "images/10.jpg"
img = tf.keras.utils.load_img(img_path, target_size=(224, 224))
img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, axis=0)
img_array = img_array / 255.0

In [4]:
model(inputs = img_array)

<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[0.27284068, 0.7271594 ]], dtype=float32)>

### Encapsulation dans un mlflow pyfunc modèle


In [5]:
import mlflow

mlflow.set_tracking_uri("http://model-platform.com/registry/vio/")
mlflow.pyfunc.log_model(
            artifact_path="custom_model",
            python_model="marker_quality_control_wrapper.py",
            artifacts={"model_path": "marker_quality_control/1"},
            registered_model_name="marker_quality_control"
        )


/home/marc-medlock/PycharmProjects/model_platform/.venv/lib/python3.12/site-packages/mlflow/pyfunc/utils/data_validation.py:186: UserWarning: Add type hints to the `predict` method to enable data validation and automatic signature inference during model logging. Check https://mlflow.org/docs/latest/model/python_model.html#type-hint-usage-in-pythonmodel for more details.
  color_warning(
/home/marc-medlock/PycharmProjects/model_platform/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025/12/03 14:11:46 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'marker_quality_control'.
2025/12/03 14:11:49 INFO mlflow.store.model_registry.abstract_store:

## Prédiction en pullant le modèle depuis la modèle plateform

In [6]:
model = mlflow.pyfunc.load_model(model_uri="models:/marker_quality_control/1")

RestException: RESOURCE_DOES_NOT_EXIST: Model Version (name=marker_quality_control, version=3) not found

In [8]:
model.predict(img)

TypeError: a bytes-like object is required, not 'JpegImageFile'

### Faire un curl sur l'endpoint déployé avec une image
curl -X POST http://model-platform.com/deploy/test/test-marker-quality-control-1-deployment-3b4041/predict \
  -F "file=@images/10.jpg"